# Calculate the biplot coordinates

If you have made it this far, give yourself a pat on the back -- that is a tricky step to retrieve the model parameters.

Next we will compute biplot coordinates so that we can visualize shiny biplots in Emperor.  This procedure will be available in qiime2 in the future -- but for now, we will have to do this calculation in a notebook.

There are only 2 parameters to tune below.  The first one is `dim`, which is the number of dimensions to approximate the conditional probability matrix (i.e. the ranks).  The second one is the scale aspect between the metabolite points and the microbe arrows.  To avoid things becoming too funky, the scale can either shrink the arrows (with scale < 1) or expand the arrows (with scale > 1).  This is mainly for aesthetic purposes.

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from biom import load_table
from skbio.stats.composition import clr, clr_inv, centralize, closure
from skbio.stats.ordination import OrdinationResults
from scipy.spatial.distance import euclidean

dim = 3      # --latent-dim
scale = 1.   # aspect ratios between metabolite points and microbe arrows

# read in the ranks.csv file we saved before.
ranks = pd.read_csv('../results/cf_output/ranks.csv', index_col=0)

# Everything below are technical details.  You shouldn't have to touch this.
ranks = ranks.apply(clr_inv).T
u, s, v = np.linalg.svd(clr(centralize(ranks)))
n = v.shape[1]
u = u[:, :dim]
s = s[:dim]
v = v[:dim, :]

u = u * np.sqrt(n-1) / scale 
v = scale * np.diag(s) @ v / np.sqrt(n-1)

# push results into Ordination object
pcids = ['PC%d' % i for i in range(len(s))]
samples = pd.DataFrame(u[:, :len(s)], index=ranks.index, columns=pcids)
features = pd.DataFrame(v.T, index=ranks.columns, columns=pcids)
eigvals = pd.Series(s, index=pcids)

features['importance'] = features.apply(lambda x: euclidean(np.zeros_like(x), x), axis=1)
features.sort_values('importance', inplace=True, ascending=False)
features.drop(['importance'], inplace=True, axis=1)

res = OrdinationResults('MultiomicsBiplot', 'Multiomics Biplot',
                        eigvals = eigvals,
                        proportion_explained = eigvals / eigvals.sum(),
                        samples=samples, features=features)

# save that shit!
res.write('../results/cf_output/omics-biplot.results')

'../results/cf_output/omics-biplot.results'

If you want to load this into an emperor biplot, you can run the following commands in a qiime2 environment

```bash
qiime tools import --input-path ../results/omics-biplot.results \
                   --output-path ../results/omics-biplot.qza \
                   --type "PCoAResults % Properties('biplot')"
```

```bash
qiime emperor biplot --i-biplot ../results/omics-biplot.qza \
                     --m-sample-metadata-file ../data/lcms_annotations.txt \
                     --m-feature-metadata-file ../data/taxonomy.tsv \
                     --p-number-of-features 10 \
                     --o-visualization ../results/emperor-omics.qza
```

Are you getting an error from this command? Turns out that you need to make sure that `sampleid` is the name of the first column in `lcms_annotations.txt` and `featureid` is the name of the first column in `taxonomy.tsv`.  That can be done as follows.

In [2]:
import os
import numpy as np
import pandas as pd

data_dir = '../data/CF'

taxonomy = pd.read_table(os.path.join(data_dir, 'taxonomy.tsv'), index_col=0)
ms_md = pd.read_table(os.path.join(data_dir, 'lcms_annotations.txt'))

ms_md.index.name = 'sampleid'
ms_md.to_csv('../results/point-metadata.txt', sep='\t')

taxonomy.index.name = 'featureid'
taxonomy.to_csv('../results/arrow-metadata.txt', sep='\t')

Are you still getting errors after this? To get around that, you can run --p-ignore-missing-samples to drop any samples that aren't shared between the metadata mapping files.

We will go more in-depth on how to create beautiful metadata to paint your Emperor biplots. But first, we will cover how to customize biplots so that you can place custom labels directly on your plot. Note that this won't be 3D or interactive -- but it allows for more customizability at the moment (but be sure to watch out for updates with Emperor, since it is under active development).

In [3]:
ranks.T.head()

,X290.0883mz60.1277,X291.0489mz61.9903,X254.1601mz62.7917,X265.1154mz65.0188,X118.0839mz71.5519,X127.0605mz78.9317,X86.0928mz97.5564,X141.9563mz99.3446,X226.1042mz146.3815,X188.0692mz152.4098,...,X716.4112mz590.2013,X812.5824mz593.0447,X343.2854mz593.0689,X741.5355mz593.2980,X434.3601mz593.5585,X412.3782mz593.7169,X439.3937mz598.8498,X461.3787mz599.1107,X460.3757mz599.3083,X438.3935mz599.3187
TACGTAGGTGGCAAGCGTTGTCCGGATTTATTGGGCGTAAAGCGAGCGCAGGCGGTTCCTTAAGTCTGATGTGAAAGCCCCCGGCTCAACCGGGGAGGGTCATTGGAAACTGGGGAACTTGAGTGCAGAAGAGGAGAGTGGAATTCCATG,0.001230,0.009182,0.005149,0.005459,0.003798,0.003523,0.004734,0.005888,0.022202,0.005883,...,0.005370,0.005991,0.004949,0.004310,0.004989,0.005669,0.004276,0.004757,0.005107,0.005710
TACGGAGGGTGCGAGCGTTAATCGGAATAACTGGGCGTAAAGGGCACGCAGGCGGTGACTTAAGTGAGGTGTGAAAGCCCCGGGCTTAACCTGGGAATTGCATTTCATACTGGGTCGCTAGAGTACTTTAGGGAGGGGTAGAATTCCACG,0.001099,0.001336,0.008822,0.008269,0.012435,0.012446,0.003789,0.007966,0.001347,0.005681,...,0.005835,0.005468,0.009051,0.007707,0.009216,0.008705,0.008762,0.007989,0.009276,0.008711
TACGTAGGTCCCGAGCGTTGTCCGGATTTATTGGGCGTAAAGCGAGCGCAGGCGGTTAGATAAGTCTGAAGTTAAAGGCTGTGGCTTAACCATAGTATGCTTTGGAAACTGTTTAACTTGAGTGCAGAAGGGGAGAGTGGAATTCCATGT,0.000002,0.035953,0.014225,0.016832,0.005955,0.005985,0.098926,0.005973,0.001798,0.050488,...,0.008974,0.050385,0.021741,0.000309,0.016209,0.016135,0.029957,0.027520,0.015741,0.015083
TACGTAGGTCCCGAGCGTTGTCCGGATTTATTGGGCGTAAAGCGAGCGCAGGCGGTTAGATAAGTCTGAAGTTAAAGGCTGTGGCTTAACCATAGTACGCTTTGGAAACTGTTTAACTTGAGTGCAAGAGGGGAGAGTGGAATTCCATGT,0.002955,0.017170,0.005722,0.007336,0.003840,0.003514,0.007954,0.005589,0.019146,0.008787,...,0.005062,0.008416,0.006219,0.002795,0.005775,0.006606,0.005764,0.005966,0.005922,0.006556
TACGTAGGTCCCGAGCGTTATCCGGATTTATTGGGCGTAAAGCGAGCGCAGGCGGTTAGATAAGTCTGAAGTTAAAGGCTGTGGCTTAACCATAGTACGCTTTGGAAACTGTTTAACTTGAGTGCAGAAGGGGAGAGTGGAATTCCATGT,0.005554,0.025121,0.006316,0.010412,0.004542,0.004075,0.010293,0.005468,0.013734,0.011383,...,0.004329,0.009852,0.007775,0.002485,0.006701,0.007647,0.007593,0.006993,0.006928,0.007500


In [4]:
res.samples

,PC0,PC1,PC2
X290.0883mz60.1277,1.340645,-0.402680,-1.179627
X291.0489mz61.9903,-0.613122,1.815497,-0.103819
X254.1601mz62.7917,-0.268177,-0.136907,0.067728
X265.1154mz65.0188,-0.484228,0.463500,-0.356947
X118.0839mz71.5519,-0.190352,-0.317677,-0.301117
X127.0605mz78.9317,-0.189637,-0.406592,-0.247114
X86.0928mz97.5564,-0.816402,0.517390,0.467442
X141.9563mz99.3446,0.020150,-0.210765,0.010557
X226.1042mz146.3815,0.690354,0.982531,0.130626
X188.0692mz152.4098,-0.632817,0.463254,0.173392
